In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x57d3c000 @  0x7fb6923f42a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import io

from torch.autograd import Variable

plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline
from torch.utils.data import Dataset

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.nn import functional
from asyncio import PriorityQueue

import time
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_CUDA = torch.cuda.is_available()

MAX_LENGTH = 30 #temp

MAX_VOCAB_SIZE = 50000

PAD_IDX = 0 
SOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3

In [0]:
en_loc = 'gdrive/My Drive/iwslt-vi-en'

In [5]:
p = PriorityQueue()
p.put_nowait((1, 'a'))
p.put_nowait((3, 'c'))
p.put_nowait((2, 'b'))
print(p.get_nowait())
print(p.get_nowait())
print(p.get_nowait())

(1, 'a')
(2, 'b')
(3, 'c')


In [0]:
import torch
from torch.nn import functional
from torch.autograd import Variable

def sequence_mask(sequence_length, max_len=None):
    if max_len is None:
        max_len = sequence_length.data.max()
    batch_size = sequence_length.size(0)
    seq_range = torch.range(0, max_len - 1).long()
    seq_range_expand = seq_range.unsqueeze(0).expand(batch_size, max_len)
    seq_range_expand = Variable(seq_range_expand)
    if sequence_length.is_cuda:
        seq_range_expand = seq_range_expand.cuda()
    seq_length_expand = (sequence_length.unsqueeze(1)
                         .expand_as(seq_range_expand))
    return seq_range_expand < seq_length_expand


def masked_cross_entropy(logits, target, length):
    length = Variable(torch.LongTensor(length))

    """
    Args:
        logits: A Variable containing a FloatTensor of size
            (batch, max_len, num_classes) which contains the
            unnormalized probability for each class.
        target: A Variable containing a LongTensor of size
            (batch, max_len) which contains the index of the true
            class for each corresponding step.
        length: A Variable containing a LongTensor of size (batch,)
            which contains the length of each data in a batch.
    Returns:
        loss: An average loss value masked by the length.
    """

    # logits_flat: (batch * max_len, num_classes)
    logits_flat = logits.view(-1, logits.size(-1))
    # log_probs_flat: (batch * max_len, num_classes)
    log_probs_flat = functional.log_softmax(logits_flat)
    # target_flat: (batch * max_len, 1)
    target_flat = target.view(-1, 1)
    # losses_flat: (batch * max_len, 1)
    losses_flat = -torch.gather(log_probs_flat, dim=1, index=target_flat)
    # losses: (batch, max_len)
    losses = losses_flat.view(*target.size())
    # mask: (batch, max_len)
    mask = sequence_mask(sequence_length=length, max_len=target.size(1))
    losses = losses * mask.float()
    loss = losses.sum() / length.float().sum()
    return loss


# Remove punctuation
def removePunctuation(s):

    to_remove = ('&lt;', '&gt;', '&amp;', '&apos;', '&quot;')
    table = str.maketrans(dict.fromkeys('.!?:,'))
    s = s.translate(table)
    for i in to_remove:
        s=s.replace(i,'')   
    s = s.strip()
    
    return s


from typing import List
from collections import Counter, namedtuple
from itertools import zip_longest

def tokenize_13a(line):
    """
    Tokenizes an input line using a relatively minimal tokenization that is however equivalent to mteval-v13a, used by WMT.
    :param line: a segment to tokenize
    :return: the tokenized line
    """

    norm = line

    # language-independent part:
    norm = norm.replace('<skipped>', '')
    norm = norm.replace('-\n', '')
    norm = norm.replace('\n', ' ')
    norm = norm.replace('&quot;', '"')
    norm = norm.replace('&amp;', '&')
    norm = norm.replace('&lt;', '<')
    norm = norm.replace('&gt;', '>')

    # language-dependent part (assuming Western languages):
    norm = " {} ".format(norm)
    norm = re.sub(r'([\{-\~\[-\` -\&\(-\+\:-\@\/])', ' \\1 ', norm)
    norm = re.sub(r'([^0-9])([\.,])', '\\1 \\2 ', norm)  # tokenize period and comma unless preceded by a digit
    norm = re.sub(r'([\.,])([^0-9])', ' \\1 \\2', norm)  # tokenize period and comma unless followed by a digit
    norm = re.sub(r'([0-9])(-)', '\\1 \\2 ', norm)  # tokenize dash when preceded by a digit
    norm = re.sub(r'\s+', ' ', norm)  # one space only between words
    norm = re.sub(r'^\s+', '', norm)  # no leading space
    norm = re.sub(r'\s+$', '', norm)  # no trailing space

    return norm

def corpus_bleu(sys_stream, ref_streams, smooth='exp', smooth_floor=0.0, force=False, lowercase=False,
                 use_effective_order=False):
    """Produces BLEU scores along with its sufficient statistics from a source against one or more references.
    :param sys_stream: The system stream (a sequence of segments)
    :param ref_streams: A list of one or more reference streams (each a sequence of segments)
    :param smooth: The smoothing method to use
    :param smooth_floor: For 'floor' smoothing, the floor to use
    :param force: Ignore data that looks already tokenized
    :param lowercase: Lowercase the data
    :param tokenize: The tokenizer to use
    :return: a BLEU object containing everything you'd want
    """

    # Add some robustness to the input arguments
    if isinstance(sys_stream, str):
        sys_stream = [sys_stream]
    if isinstance(ref_streams, str):
        ref_streams = [[ref_streams]]

    sys_len = 0
    ref_len = 0

    correct = [0 for n in range(NGRAM_ORDER)]
    total = [0 for n in range(NGRAM_ORDER)]
    

    # look for already-tokenized sentences
    tokenized_count = 0

    fhs = [sys_stream] + ref_streams
    for lines in zip_longest(*fhs):
        if None in lines:
            raise EOFError("Source and reference streams have different lengths!")

        if lowercase:
            lines = [x.lower() for x in lines]
            
        tokenize= 'tokenize_13a'    

        if not (force or tokenize == 'none') and lines[0].rstrip().endswith(' .'):
            tokenized_count += 1

            if tokenized_count == 100:
                logging.warning('That\'s 100 lines that end in a tokenized period (\'.\')')
                logging.warning('It looks like you forgot to detokenize your test data, which may hurt your score.')
                logging.warning('If you insist your data is detokenized, or don\'t care, you can suppress this message with \'--force\'.')

        output, *refs = [tokenize_13a(x.rstrip()) for x in lines]
        

        ref_ngrams, closest_diff, closest_len = ref_stats(output, refs)
        

        sys_len += len(output.split())
        ref_len += closest_len

        sys_ngrams = extract_ngrams(output)
        for ngram in sys_ngrams.keys():
            n = len(ngram.split())
            correct[n-1] += min(sys_ngrams[ngram], ref_ngrams.get(ngram, 0))
            total[n-1] += sys_ngrams[ngram]
            

    return compute_bleu(correct, total, sys_len, ref_len, smooth, smooth_floor, use_effective_order)
  
  
# n-gram order. Don't change this.
NGRAM_ORDER = 4
  
def compute_bleu(correct: List[int], total: List[int], sys_len: int, ref_len: int, smooth = 'none', smooth_floor = 0.01,
                 use_effective_order = False):
    """Computes BLEU score from its sufficient statistics. Adds smoothing.
    :param correct: List of counts of correct ngrams, 1 <= n <= NGRAM_ORDER
    :param total: List of counts of total ngrams, 1 <= n <= NGRAM_ORDER
    :param sys_len: The cumulative system length
    :param ref_len: The cumulative reference length
    :param smooth: The smoothing method to use
    :param smooth_floor: The smoothing value added, if smooth method 'floor' is used
    :param use_effective_order: Use effective order.
    :return: A BLEU object with the score (100-based) and other statistics.
    """

    precisions = [0 for x in range(NGRAM_ORDER)]

    smooth_mteval = 1.
    effective_order = NGRAM_ORDER
    for n in range(NGRAM_ORDER):
        if total[n] == 0:
            break

        if use_effective_order:
            effective_order = n + 1

        if correct[n] == 0:
            if smooth == 'exp':
                smooth_mteval *= 2
                precisions[n] = 100. / (smooth_mteval * total[n])
            elif smooth == 'floor':
                precisions[n] = 100. * smooth_floor / total[n]
        else:
            precisions[n] = 100. * correct[n] / total[n]

    # If the system guesses no i-grams, 1 <= i <= NGRAM_ORDER, the BLEU score is 0 (technically undefined).
    # This is a problem for sentence-level BLEU or a corpus of short sentences, where systems will get no credit
    # if sentence lengths fall under the NGRAM_ORDER threshold. This fix scales NGRAM_ORDER to the observed
    # maximum order. It is only available through the API and off by default

    brevity_penalty = 1.0
    if sys_len < ref_len:
        brevity_penalty = math.exp(1 - ref_len / sys_len) if sys_len > 0 else 0.0
        

    bleu = brevity_penalty * math.exp(sum(map(my_log, precisions[:effective_order])) / effective_order)

    return bleu 
  
  
def ref_stats(output, refs):
    ngrams = Counter()
    closest_diff = None
    closest_len = None
    for ref in refs:
        tokens = ref.split()
        reflen = len(tokens)
        diff = abs(len(output.split()) - reflen)
        if closest_diff is None or diff < closest_diff:
            closest_diff = diff
            closest_len = reflen
        elif diff == closest_diff:
            if reflen < closest_len:
                closest_len = reflen

        ngrams_ref = extract_ngrams(ref)
        for ngram in ngrams_ref.keys():
            ngrams[ngram] = max(ngrams[ngram], ngrams_ref[ngram])

    return ngrams, closest_diff, closest_len
  
  
def extract_ngrams(line, min_order=1, max_order=NGRAM_ORDER) -> Counter:
    """Extracts all the ngrams (1 <= n <= NGRAM_ORDER) from a sequence of tokens.
    :param line: a segment containing a sequence of words
    :param max_order: collect n-grams from 1<=n<=max
    :return: a dictionary containing ngrams and counts
    """

    ngrams = Counter()
    tokens = line.split()
    for n in range(min_order, max_order + 1):
        for i in range(0, len(tokens) - n + 1):
            ngram = ' '.join(tokens[i: i + n])
            ngrams[ngram] += 1

    return ngrams  

def my_log(num):
    """
    Floors the log function
    :param num: the number
    :return: log(num) floored to a very low number
    """

    if num == 0.0:
        return -9999999999
    return math.log(num)
  

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        #self.word2count = {}
        self.index2word = {}
        self.n_words = 4  # Count SOS and EOS and Pad
        self.all_words = []
        
    def build_vocab(self, embedding, max_vocab_size = 10000):

    # save index 1 for unk and 0 for pad
    # Returns:
    # id2token: list of tokens, where id2token[i] returns token that corresponds to token i
    # token2id: dictionary where keys represent tokens and corresponding values represent indices
    #all_tokens = [item for sublist in all_tokens for item in sublist]
    #max_len = max([len(word) for word in all_tokens])
    
    
        unique_words = list(embedding.keys())[0:max_vocab_size]

        index2word =  unique_words #list of words available in embedding
        index2word = ['<pad>', '<sos>', '<eos>','<unk>'] + index2word #add pad and unknown to the beginning

        word2index = dict(zip(unique_words, range(4,4+len(unique_words)))) # dictionary of words and indices 
        word2index['<pad>'] = PAD_IDX  #add pad symbol to the dictionary
        word2index['<unk>'] = UNK_IDX  #add unkown symbol to the dictionary
        word2index['<eos>'] = EOS_IDX
        word2index['<sos>'] = SOS_IDX
        
        self.word2index = word2index
        self.index2word = index2word
        self.n_words = len(self.word2index)

        return word2index, index2word          

            
def remove_blanks(pair):
    '''Remove empty lines'''
    if len(pair[0]) == 0 and len(pair[1]) == 0:
        return False
    else:
        return True
    
def set_max_length(pair, max_length=MAX_LENGTH):
    if len(pair[0].split(' ')) > max_length or len(pair[1].split(' '))>max_length:
        return False
    else:
        return True
    
def readLangs(filename1, filename2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    with open(filename1, encoding='utf-8') as f:
        lines1 = f.read().strip().split('\n')
        
    with open(filename2, encoding='utf-8') as f:
        lines2 = f.read().strip().split('\n')   
        
    # Remove punctuation
    lines1 = [removePunctuation(l) for l in lines1]
    lines2 = [removePunctuation(l) for l in lines2]
              
    # Reverse pairs, make Lang instances
    if reverse: #change from english->french to french->english for example
        pairs =list(zip(lines2, lines1))
        input_lang = Lang(filename2[-2:]) #take last two letters
        output_lang = Lang(filename1[-2:])
    else:
        pairs =list(zip(lines1, lines2))
        input_lang = Lang(filename1[-2:])
        output_lang = Lang(filename2[-2:])
            
        

    pairs = list(filter(remove_blanks, pairs))  
    pairs = list(filter(set_max_length, pairs))

    return input_lang, output_lang, pairs 


def prepareData(lang1, lang2, embedding_in, embedding_out, num_sent=None, reverse=False):
    
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    
    pairs = pairs[:num_sent]
    print("Read %s sentence pairs" % len(pairs))
    
    print("Counting words...")
        
    #print(type(embedding_in))    
    input_lang.build_vocab(embedding_in)
    output_lang.build_vocab(embedding_out)
        
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs



class VocabDataset(Dataset):
    """
    Class that represents a train/validation/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, data_tuple, word2id_lang1, word2id_lang2):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.data_list1, self.data_list2 = zip(*data_tuple)
        assert (len(self.data_list1) == len(self.data_list2))
        self.word2id1 = word2id_lang1
        self.word2id2 = word2id_lang2

    def __len__(self):
        return len(self.data_list1)

    def __getitem__(self, key):
        """
        Triggered when you call dataset[i]
        """
        
        input_sentence = [self.word2id1[c] if c in self.word2id1.keys() 
                         else UNK_IDX for c in self.data_list1[key].split()][:MAX_LENGTH-1]
        input_sentence.append(EOS_IDX)
                                                                   
        output_sentence = [self.word2id2[c] if c in self.word2id2.keys() 
                          else UNK_IDX for c in self.data_list2[key].split()][:MAX_LENGTH-1]
        output_sentence.append(EOS_IDX)

        return [input_sentence, output_sentence, len(input_sentence), len(output_sentence)]
    

def vocab_collate_func(batch):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    data_list1 = []
    data_list2 = []
    length_list1 = []
    length_list2 = []
     
    # padding
    for datum in batch:
        x1 = datum[0]
        x2 = datum[1]
        len1 = datum[2]
        len2 = datum[3]
        
        length_list1.append(len1)
        length_list2.append(len2)
        #Pad first sentences
        padded_vec1 = np.pad(np.array(x1),
                                pad_width=((0,MAX_LENGTH-len1)),
                                mode="constant", constant_values=0)
        data_list1.append(padded_vec1)
        
        #Pad second sentences
        padded_vec2 = np.pad(np.array(x2),
                        pad_width=((0,MAX_LENGTH-len2)),
                        mode="constant", constant_values=0)
        data_list2.append(padded_vec2)
        
    data_list1 = np.array(data_list1)
    data_list2 = np.array(data_list2)
    length_list1 = np.array(length_list1)
    lenth_list2 = np.array(length_list2)
    
    return [torch.from_numpy(np.array(data_list1)), 
            torch.from_numpy(np.array(data_list2)),
            torch.LongTensor(length_list1), 
            torch.LongTensor(length_list2)]




      
class EncoderCNN(nn.Module):
    def __init__(self, hidden_size, emb_weights, num_layers, kernel_size):
        super(EncoderCNN, self).__init__()

        self.num_layers, self.hidden_size = num_layers, hidden_size
        self.num_embeddings, self.embedding_dim = emb_weights.size()
        self.embedding = nn.Embedding.from_pretrained(emb_weights,freeze=True)
   
        self.conv1 = nn.Conv1d(self.embedding_dim, hidden_size, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.linear = nn.Linear(hidden_size,hidden_size)

    def forward(self, input_seqs, input_lengths, hidden=None):

        embed = self.embedding(input_seqs.to(device))
        batch_size,seq_len=input_seqs.size()
        #First sentence
        hidden = self.conv1(embed.transpose(1,2)).transpose(1,2)

        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(batch_size, seq_len, hidden.size(-1))

        hidden = self.conv2(hidden.transpose(1,2)).transpose(1,2)

        hidden = F.relu(hidden.contiguous().view(-1, hidden.size(-1))).view(batch_size, seq_len, hidden.size(-1))
        hidden=torch.sum(hidden, dim=0).view(1,seq_len,hidden.size(-1))

        hidden=self.linear(hidden)

        #logits = self.linear(hiddenfinal)
        return 1,hidden




class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, vocab_size, emb_weights):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        weight_size = emb_weights.size()[1]
        self.embedding = nn.Embedding(vocab_size, weight_size, padding_idx=0).from_pretrained(emb_weights, 
                                                                freeze=True).to(device)
        
        self.gru = nn.GRU(weight_size, hidden_size)
        self.out = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, inp, hidden):
        embedded = self.embedding(inp.to(device)).unsqueeze(0) #so that we have 1 x batch x hidden
        #print('embedded', embedded.size())
        output = F.relu(embedded)
        #print('after relu', output.size())
        #print('hidden size', hidden.size())
#         print('decoder output size', output.size())
#         print('decoder hidden size', hidden.size())
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden


class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self):
        return self.logp / float(self.leng - 1 + 1e-6)

    
def train(inputs, input_lengths, targets, target_lengths, 
          encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH,
         teacher_forcing_ratio=0.5):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 #
    batch_size = inputs.size()[1]
    #print('input size', inputs.size())
    #print('batch size', batch_size)
    max_targ_len = max_length

    # Run words through encoder
    _, encoder_hidden = encoder(inputs, input_lengths, None)

    
    # Prepare input and output variables
    decoder_input = torch.LongTensor([SOS_IDX] * batch_size).to(device)
    decoder_hidden = encoder_hidden#[:1] # Use last (forward) hidden state from encoder
    
    #print('time 1 size', decoder_input.size())
    #print('time 1 hidden size', decoder_hidden.size())
    
    #randomly use teacher forcing or not
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Run through decoder one time step at a time using TEACHER FORCING=1.0
    all_decoder_outputs = Variable(torch.zeros(max_targ_len, batch_size, output_lang.n_words))
    


    if use_teacher_forcing:
        for t in range(max_targ_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            all_decoder_outputs[t] = decoder_output
            decoder_input = targets[t]
            
    else:
        # Without teacher forcing: use its own predictions as the next input without beam
        for di in range(max_targ_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)

            decoder_input = topi.squeeze().detach()  # detach from history as input
            
            all_decoder_outputs[di] = decoder_output

                
    loss = masked_cross_entropy(
    all_decoder_outputs.transpose(0, 1).contiguous(),
    targets.transpose(0, 1).contiguous(),
    target_lengths)
        
    loss.backward()

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item()

def trainIters(loader, encoder, decoder, n_iters, print_every=1000, plot_every=100, validate_every=1,
               learning_rate=0.01,
              teacher_forcing_ratio=1, beam_width=3, beam_search=False):
    
    start = time.time()
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    print_loss_total = 0
    plot_loss_total = 0
    plot_losses = []
    val_bleu = []

    counter = 0
    epoch = 0

    while epoch < n_iters:
        epoch += 1

        # Get training data for this cycle
        for i, (source, target, lengths1, lengths2) in enumerate(loader):

            counter += 1

            # Run the train function
            loss = train(
                source.long().transpose(0,1), lengths1, target.long().transpose(0,1), lengths2,
                encoder, decoder,
                encoder_optimizer, decoder_optimizer, criterion, teacher_forcing_ratio=teacher_forcing_ratio
            )

            # Keep track of loss
            print_loss_total += loss
            plot_loss_total += loss


            if counter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_iters), epoch, 
                                                       epoch / n_iters * 100, print_loss_avg)
                print(print_summary)


            if counter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
                
                
            if counter % validate_every == 0:
                bleu = validate(encoder, decoder, val_loader, beam_width, beam_search)
                val_bleu.append(bleu)
                print('Validation BLEU', bleu)

    showPlot(plot_losses)
    return plot_losses, val_bleu


def evaluate(encoder, decoder, sentence, input_lengths, translated, beam_width=3, beam_search=True, 
             max_length=MAX_LENGTH):
    """
    Function that generate translation.
    First, feed the source sentence into the encoder and obtain the hidden states from encoder.
    Secondly, feed the hidden states into the decoder and unfold the outputs from the decoder.
    Lastly, for each outputs from the decoder, collect the corresponding words in the target language's vocabulary.
    And collect the attention for each output words.
    @param encoder: the encoder network
    @param decoder: the decoder network
    @param sentence: string, a sentence in source language to be translated
    @param max_length: the max # of words that the decoder can return
    @output decoded_words: a list of words in target language
    @output decoder_attentions: a list of vector, each of which sums up to 1.0
    """    
    # process input sentence
    with torch.no_grad():
        input_tensor = sentence.transpose(0,1)
        input_length = sentence.size()[0]
        
        # encode the source lanugage
        encoder_output, encoder_hidden = encoder(input_tensor, input_lengths, None)

        decoder_input = torch.tensor([SOS_IDX], device=device)  # SOS
        decoder_hidden = encoder_hidden[:1] # Use last (forward) hidden state from encoder 
        # output of this function
        decoded_words = ''

        for di in range(max_length):
            # for each time step, the decoder network takes two inputs: previous outputs and the previous hidden states
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            # hint: print out decoder_output and decoder_attention
            # TODO: add your code here to populate decoded_words and decoder_attentions
            # TODO: do this in 2 ways discussed in class: greedy & beam_search
            
            if beam_search:
                topk = 1 #for beam search
                # Number of sentence to generate
                endnodes = []
                number_required = min((topk + 1), topk - len(endnodes))

                # starting node -  hidden vector, previous node, word id, logp, length
                node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
                nodes = PriorityQueue()

                # start the queue
                nodes.put_nowait((-node.eval(), node))
                qsize = 1

                # start beam search
                while True:
                    # give up when decoding takes too long
                    if qsize > 2000: break

                    # fetch the best node
                    score, n = nodes.get_nowait()
                    decoder_input = n.wordid
                    decoder_hidden = n.h

                    if n.wordid.item() == EOS_IDX and n.prevNode != None:
                        endnodes.append((score, n))
                        # if we reached maximum # of sentences required
                        if len(endnodes) >= number_required:
                            break
                        else:
                            continue

                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)

                    log_prob, indexes = torch.topk(decoder_output, beam_width)
                    nextnodes = []

                    for new_k in range(beam_width):
                        decoded_t = indexes[0][new_k].view(1, -1)
                        log_p = log_prob[0][new_k].item()

                        node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                        score = -node.eval()
                        nextnodes.append((score, node))

                    # put them into queue
                    for i in range(len(nextnodes)):
                        score, nn = nextnodes[i]
                        nodes.put_nowait((score, nn))
                        # increase qsize
                    qsize += len(nextnodes) - 1

                # choose nbest paths, back trace them
                if len(endnodes) == 0:
                    endnodes = [nodes.get_nowait() for _ in range(topk)]

                best_beam = []
                for score, n in sorted(endnodes, key=operator.itemgetter(0)):
                    utterance = []
                    utterance.append(n.wordid)
                    # back trace
                    while n.prevNode != None:
                        n = n.prevNode
                        utterance.append(n.wordid)

                    utterance = utterance[::-1]
                    best_beam.append(utterance)

                all_decoder_outputs[t] = best_beam
                decoder_input = targets[t]
            
            # GREEDY
            else:
                topv, topi = decoder_output.data.topk(1) 

                if topi.item() == EOS_IDX:
                    #decoded_words.append('<EOS>')
                    break

                else:
                    if topi.item() not in [SOS_IDX, EOS_IDX, UNK_IDX, PAD_IDX]:
                        decoded_words = decoded_words + ' ' + output_lang.index2word[topi.item()]

                decoder_input = topi[0].detach()
        
        translation = ''
        for i in translated: #expected translation
            if i.item() not in [SOS_IDX, EOS_IDX, UNK_IDX, PAD_IDX]:
                translation = translation + ' ' + output_lang_v.index2word[i.item()]

        return decoded_words, translation
    
    
def evaluate_batch(loader, encoder, decoder, beam_width=3, beam_search=True):
    
    decoded_sentences = []
    actual_sentences = []
    
    for i, (source, target, lengths1, lengths2) in enumerate(loader):
        #iterate over batch
        
        for n in range(len(source)):
            # Go sentence by sentence
            
            decoded, actual = evaluate(encoder, decoder, source[n].unsqueeze(0), lengths1[n], target[n],
                                       beam_width, beam_search)
            decoded_sentences.append(decoded)
            actual_sentences.append(actual)
            
    return decoded_sentences, actual_sentences


def validate(encoder, decoder, val_loader, beam_width, beam_search):
    decoded_sentences, actual_sentences = evaluate_batch(val_loader, encoder, decoder, beam_width, beam_search)
    bleu = evaluate_bleu(decoded_sentences, actual_sentences)
    
    return bleu


def evaluate_bleu(translation_list, reference_list):
    
    return corpus_bleu(translation_list, [reference_list])

#Plot results
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    

def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))    

def load_embedding(fname, max_count=None):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    counter=0
    for line in fin:
        counter+=1
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
        if counter==max_count:
            break
    return data

In [0]:
### VIETNAMESE
#Read in pretrained embedding vectors - subset for now
embeddings_map_vi = load_embedding(en_loc + '/cc.vi.300.vec', max_count=50000)

#Convert embedding values to lists
embeddings_vi = {}

for key, value in embeddings_map_vi.items():
    embeddings_vi[key] = list(value)
    
### ENGLISH    
#Read in pretrained embedding vectors - subset for now
embeddings_map_en = load_embedding(en_loc + '/wiki-news-300d-1M.vec', max_count=50000)

#Convert embedding values to lists
embeddings_en = {}

for key, value in embeddings_map_en.items():
    embeddings_en[key] = list(value)

In [0]:
def embed_to_tensor(embeddings):
    y=np.array([np.array(list(xi)) for xi in embeddings.values()])
    padding = np.zeros((1, y.shape[1]))
    unknown = np.random.rand(1, y.shape[1]) # to account for Padding and Unknown
    sos = np.random.rand(1, y.shape[1])
    eos = np.random.rand(1, y.shape[1])
    full_size = np.concatenate([padding, sos, eos, unknown, y], axis=0)
    emb_weights = torch.from_numpy(full_size)
    
    return emb_weights

In [0]:
embed_vi_tensor = embed_to_tensor(embeddings_vi).float()
embed_en_tensor = embed_to_tensor(embeddings_en).float()

In [154]:
input_lang, output_lang, pairs = prepareData(en_loc+'/train.tok.vi', en_loc+'/train.tok.en', 
                                             embedding_in = embeddings_vi,
                                             embedding_out = embeddings_en, num_sent=100)

Reading lines...
Read 100 sentence pairs
Counting words...
Counted words:
vi 10004
en 10004


In [155]:
input_lang_v, output_lang_v, pairs_v = prepareData(en_loc+'/dev.tok.vi', en_loc+'/dev.tok.en', 
                                                   embedding_in = embeddings_vi,
                                                   embedding_out = embeddings_en,
                                                   num_sent=100)

Reading lines...
Read 100 sentence pairs
Counting words...
Counted words:
vi 10004
en 10004


In [156]:
pairs_v[0:5]

[('Khi tôi còn nhỏ  Tôi nghĩ rằng BắcTriều Tiên là đất_nước tốt nhất trên thế_giới và tôi thường hát bài " Chúng_ta chẳng có gì phải ghen_tị  "',
  'When I was little  I thought my country was the best on the planet  and I grew up singing a song called  Nothing To Envy'),
 ('Tôi đã rất tự_hào về đất_nước tôi', 'And I was very proud'),
 ('Khi tôi lên 7  tôi chứng_kiến cảnh người_ta xử_bắn công_khai lần đầu_tiên trong đời  nhưng tôi vẫn nghĩ cuộc_sống của mình ở đây là hoàn_toàn bình_thường',
  'When I was seven years old  I saw my first public execution  but I thought my life in North Korea was normal'),
 ('Gia_đình của tôi không nghèo  và bản_thân tôi thì chưa_từng phải chịu đói',
  'My family was not poor  and myself  I had never experienced hunger'),
 ('Nhưng vào một ngày của năm 1995  mẹ tôi mang về nhà một lá thư_từ một người chị_em cùng chỗ làm với mẹ',
  'But one day  in 1995  my mom brought home a letter from a coworker s sister')]

In [0]:
BATCH_SIZE=32
hidden_size=256

train_dataset = VocabDataset(pairs, input_lang.word2index, output_lang.word2index)
# 1 batch input dimension: num_sentences x max sentence length
# 1 batch: source_sentences, target_sentences, source_lengths, target_lengths
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=True)

val_dataset = VocabDataset(pairs_v, input_lang.word2index, output_lang.word2index)
# 1 batch input dimension: num_sentences x max sentence length
# 1 batch: source_sentences, target_sentences, source_lengths, target_lengths
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=BATCH_SIZE,
                                           collate_fn=vocab_collate_func,
                                           shuffle=False)

encoder = EncoderCNN(hidden_size = hidden_size,
                    emb_weights = embed_vi_tensor,num_layers=1, kernel_size=2).to(device)

decoder = DecoderRNN(hidden_size = hidden_size, vocab_size = output_lang.n_words,
                    emb_weights = embed_en_tensor).to(device)

plot_losses, validation_bleu = trainIters(train_loader, encoder, decoder, n_iters=2000, 
                         print_every=50, plot_every=100, validate_every = 50, learning_rate=0.01, 
                                          teacher_forcing_ratio=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  if __name__ == '__main__':


0m 29s (- 74m 24s) (13 0%) 9.2057
Validation BLEU 0.006207471707676466
1m 1s (- 81m 32s) (25 1%) 9.1617
Validation BLEU 0.030196857919202756
1m 35s (- 81m 50s) (38 1%) 9.1070
Validation BLEU 0.0
2m 4s (- 80m 44s) (50 2%) 9.0167
Validation BLEU 0.0
2m 33s (- 78m 49s) (63 3%) 8.4508
Validation BLEU 0.0
3m 2s (- 78m 12s) (75 3%) 6.7530
Validation BLEU 0.0
3m 32s (- 76m 58s) (88 4%) 6.1122
Validation BLEU 5.54063825288447e-07
4m 1s (- 76m 33s) (100 5%) 5.8025
Validation BLEU 0.02728725215554096
4m 31s (- 75m 39s) (113 5%) 5.6298
Validation BLEU 0.042887991932221516
5m 1s (- 75m 17s) (125 6%) 5.4932
Validation BLEU 0.0
5m 31s (- 74m 30s) (138 6%) 5.2798
Validation BLEU 0.020342174257477485
6m 0s (- 74m 9s) (150 7%) 5.1662
Validation BLEU 0.0
6m 31s (- 73m 28s) (163 8%) 5.1371
Validation BLEU 0.10774304576287853
7m 0s (- 73m 6s) (175 8%) 5.0551
Validation BLEU 0.07078445743389188
7m 31s (- 72m 28s) (188 9%) 5.0602
Validation BLEU 0.1543924013249309
8m 0s (- 72m 4s) (200 10%) 4.9723
Validatio

In [0]:
embed_vi_tensor.size()

In [0]:
decoded, actual = evaluate_batch(val_loader, encoder, decoder)

for i in zip(decoded, actual):
    if i == 10:
        break
    print('\n')
    print('Expected:', i[1])
    print('Actual:' ,i[0])

In [0]:
evaluate_bleu(decoded, actual)